# **Điểm chuẩn - Preprocessing**

Preprocessing data điểm chuẩn đã crawl về từ web [huongnghiep.hocmai.vn](https://huongnghiep.hocmai.vn/diem-chuan/)

## **Intro:**
- Bộ dữ liệu raw là các file csv như sau:  

    ```
    CRAWLER-HOCMAI  
    └── data  
        └── full  
            ├── diemchuan2018_demo.csv  
            ├── diemchuan2019_full.csv  
            ├── diemchuan2020_full.csv  
            ├── diemchuan2021_full.csv  
            ├── diemchuan2022_full.csv  
            ├── diemchuan2023_full.csv  
    ```
- Tương ứng mỗi file là dữ liệu điểm chuẩn của 1 năm.
    - 2 file `full` là dữ liệu đã được crawled đầy đủ.
    - 4 file `demo` là dữ liệu ***gần như*** đầy đủ, vì trong quá trình crawl có bị missing một vài trường. Sẽ cố gắng cập nhật đầy đủ sớm hơn.
- Cấu trúc dữ liệu mỗi file gồm 7 cột `university_code`, `university_name`, `major_code`, `major_name`, `subject_group`, `point`, `note`, `year`. 

    Một mẫu dữ liệu minh hoạ như sau: 
    
| university_code | university_name                 | major_code | major_name               | subject_group       | point | note             | year |
|-----------------|---------------------------------|------------|--------------------------|---------------------|-------|------------------|------|
| UFL-HU          | Đại Học Ngoại Ngữ – Đại Học Huế | 7140234    | Sư phạm Tiếng Trung Quốc | D01, D15, D04, D45  | 24.85 | Tốt nghiệp THPT  | 2023 |
| UFL-HU          | Đại Học Ngoại Ngữ – Đại Học Huế | 7140233    | Sư phạm Tiếng Pháp       | D01, D15, D03, D44  | 19    | Tốt nghiệp THPT  | 2023 |
| UFL-HU          | Đại Học Ngoại Ngữ – Đại Học Huế | 7220204    | Ngôn ngữ Trung Quốc      | D01, D15, D04, D45  | 23    | Tốt nghiệp THPT  | 2023 |
| UFL-HU          | Đại Học Ngoại Ngữ – Đại Học Huế | 7220210    | Ngôn ngữ Hàn Quốc        | D01, D14, D15       | 22.5  | Tốt nghiệp THPT  | 2023 |



## **Target**
- Kiểm tra tính đầy đủ của dữ liệu (về số trường đại học).

- Kiểm tra thiếu dữ liệu của từng dòng.

- Kiểm tra trùng lặp.

- **Phân tách dữ liệu cho từng loại điểm**


## **Start here**

### **import các thư viện**

In [274]:
import pandas as pd
import numpy as np
import os

### **Chỉ định đường dẫn thư mục**

In [275]:
# Directory containing the CSV files
directory = './data/full'


### **Đọc các file csv vào df**

In [276]:
# Dictionary to store DataFrames
dfs = {}
raw = []

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Construct full file path
        file_path = os.path.join(directory, filename)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Store the DataFrame in the dictionary with the filename (without .csv) as the key
        dfs[filename.split('_')[0]] = df
        raw.append(df)

# Display the keys of the dictionary to confirm the DataFrames are loaded
dfs.keys()


dict_keys(['diemchuan2018', 'diemchuan2019', 'diemchuan2020', 'diemchuan2021', 'diemchuan2022', 'diemchuan2023'])

In [277]:
shapes = {name: df.shape for name, df in dfs.items()}
shapes

{'diemchuan2018': (4287, 8),
 'diemchuan2019': (5706, 8),
 'diemchuan2020': (4924, 8),
 'diemchuan2021': (6560, 8),
 'diemchuan2022': (7991, 8),
 'diemchuan2023': (7947, 8)}

In [278]:
full_df = pd.concat(raw, ignore_index=True)
full_df.head()

,university_code,university_name,major_code,major_name,subject_group,point,note,year
0,HMU,Đại học Thủ Đô Hà Nội,7140201,Giáo dục Mầm non,"M01, M09",37.58,TTNV ≤ 1Hệ đại học,2018
1,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7210208,ĐH Piano,N00,32.00,Năng khiếu nhân 2,2018
2,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7229042,Quản lý văn hoá,R00,32.00,Liên thông (Năng khiếu nhân 2),2018
3,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7140222,Sư phạm Mỹ thuật,H00,32.00,Liên thông (Năng khiếu nhân 2),2018
4,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7210403,Thiết kế đồ họa,H00,32.00,Năng khiếu nhân 2,2018


In [279]:
full_df.shape

(37415, 8)

In [280]:
uni_info = pd.read_csv("./data/university/university.csv")
uni_info.drop(columns='url',inplace=True)

In [281]:
full_df = pd.merge(full_df, uni_info, how="left", on=["university_code", "university_name"])
full_df

,university_code,university_name,major_code,major_name,subject_group,point,note,year,province,region,admission_code,institution_type
0,HMU,Đại học Thủ Đô Hà Nội,7140201,Giáo dục Mầm non,"M01, M09",37.58,TTNV ≤ 1Hệ đại học,2018,Hà Nội,Miền Bắc,HNM,Công lập
1,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7210208,ĐH Piano,N00,32.00,Năng khiếu nhân 2,2018,Hà Nội,Miền Bắc,GNT,Công lập
2,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7229042,Quản lý văn hoá,R00,32.00,Liên thông (Năng khiếu nhân 2),2018,Hà Nội,Miền Bắc,GNT,Công lập
3,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7140222,Sư phạm Mỹ thuật,H00,32.00,Liên thông (Năng khiếu nhân 2),2018,Hà Nội,Miền Bắc,GNT,Công lập
4,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7210403,Thiết kế đồ họa,H00,32.00,Năng khiếu nhân 2,2018,Hà Nội,Miền Bắc,GNT,Công lập
...,...,...,...,...,...,...,...,...,...,...,...,...
37410,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,DGNLHCM,761.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,Công lập
37411,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,DGNLHCM,648.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,Công lập
37412,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,DGNLHCM,643.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,Công lập
37413,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,DGNLHCM,613.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,Công lập


#### Đổi tên các cột lại thành tiếng Việt

In [282]:
# Dictionary to map old column names to new column names
column_mapping = {
    'province': 'Tỉnh/ Thành phố',
    'region': "Khu vực",
    'admission_code': 'Mã trường xét tuyển',
    'university_code': 'Mã Trường',
    'university_name': 'Tên Trường',
    'major_code': 'Mã xét tuyển',
    'major_name': 'Tên Ngành',
    'subject_group': 'Tổ hợp',
    'point': 'Điểm chuẩn',
    'note': 'Ghi chú',
    'year': 'Năm',
    'institution_type': 'Loại đơn vị'
}

# Rename columns for each DataFrame in the dictionary
for name, df in dfs.items():
    df.rename(columns=column_mapping, inplace=True)

full_df.rename(columns=column_mapping, inplace=True)
full_df.head()

,Mã Trường,Tên Trường,Mã xét tuyển,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Tỉnh/ Thành phố,Khu vực,Mã trường xét tuyển,Loại đơn vị
0,HMU,Đại học Thủ Đô Hà Nội,7140201,Giáo dục Mầm non,"M01, M09",37.58,TTNV ≤ 1Hệ đại học,2018,Hà Nội,Miền Bắc,HNM,Công lập
1,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7210208,ĐH Piano,N00,32.00,Năng khiếu nhân 2,2018,Hà Nội,Miền Bắc,GNT,Công lập
2,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7229042,Quản lý văn hoá,R00,32.00,Liên thông (Năng khiếu nhân 2),2018,Hà Nội,Miền Bắc,GNT,Công lập
3,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7140222,Sư phạm Mỹ thuật,H00,32.00,Liên thông (Năng khiếu nhân 2),2018,Hà Nội,Miền Bắc,GNT,Công lập
4,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7210403,Thiết kế đồ họa,H00,32.00,Năng khiếu nhân 2,2018,Hà Nội,Miền Bắc,GNT,Công lập


In [283]:
full_df = full_df[[ 'Tỉnh/ Thành phố', 'Khu vực', 'Mã trường xét tuyển','Mã Trường', 'Tên Trường','Mã xét tuyển', 'Tên Ngành', 'Tổ hợp', 'Điểm chuẩn', 'Ghi chú', 'Năm', 'Loại đơn vị']]

### **Khám phá dữ liệu**

#### 1. Mỗi dòng có ý nghĩa gì?

In [284]:
full_df.sample(5)

,Tỉnh/ Thành phố,Khu vực,Mã trường xét tuyển,Mã Trường,Tên Trường,Mã xét tuyển,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị
10768,Hà Nội,Miền Bắc,DCN,HAUI,Đại Học Công Nghiệp Hà Nội,7480103,Kỹ thuật phần mềm,"A00, A01",24.3,"TCP1: Toán > 8.8, TCp2: Toán = 8.8, NV<=1",2020,Công lập
24271,Đà Lạt,Miền Trung - Tây Nguyên,DYD,YU,Đại Học Yersin Đà Lạt,7580101,Kiến trúc,DGNLHCM,600.0,NaN,2022,Dân lập
14260,Hà Nội,Miền Bắc,HVN,VNUA,Học Viện Nông Nghiệp Việt Nam,HVN21,Logistics và Quản lý chuỗi cung ứng,"A00, D01, C20, A09, D01",18.0,NaN,2020,Công lập
204,Hà Nội,Miền Bắc,KQH,MTA,Học Viện Kĩ Thuật Quân Sự - Hệ Quân sự,7860220,Chỉ huy tham mưu thông tin,"A00, A01",25.1,Thí sinh Nữ miền Bắc,2018,Công lập
22816,Hồ Chí Minh,Miền Nam,DVH,VHU,Đại Học Văn Hiến,7510605,Logistics và Quản lý chuỗi cung ứng,"A00, A01, D01, C04",22.0,Điểm thi TN THPT,2022,Dân lập


Mỗi dòng là thông tin về điểm chuẩn của một ngành, tại một trường, vào một năm cụ thể.

##### Kiểm tra trùng lặp

In [285]:
if len(full_df[full_df.duplicated()]) == 0:
    print(f'Không có dòng nào trùng lặp trong dữ liệu.')
else:
    print(f'Tồn tại {len(full_df[full_df.duplicated()])} dòng bị trùng lặp trong dữ liệu.')

Tồn tại 16 dòng bị trùng lặp trong dữ liệu.


Xuất các dòng trùng lặp ra file csv (Để coi chơi thôi chứ cũng drop à :v)

In [286]:
full_df[full_df.duplicated()].to_csv('./data/dup.csv',  encoding='utf-8-sig')

Loại bỏ các dòng trùng

In [287]:
full_df.drop_duplicates(inplace=True)

C:\Users\letra\AppData\Local\Temp\ipykernel_89344\2742331558.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df.drop_duplicates(inplace=True)


Kiểm tra lại

In [288]:
if len(full_df[full_df.duplicated()]) == 0:
    print(f'Không có dòng nào trùng lặp trong dữ liệu.')
else:
    print(f'Tồn tại {len(full_df[full_df.duplicated()])} dòng bị trùng lặp trong dữ liệu.')

Không có dòng nào trùng lặp trong dữ liệu.


#### 2. Mỗi cột có ý nghĩa gì?

Các cột và thông tin tương ứng:
- `Mã Trường`: Tên viết tắt của trường (Không phải mã trên hệ thống của bộ).

- `Tên Trường`: Tên của Trường đại học.

- `	Mã xét tuyển`: Mã ngành xét tuyển.

- `	Tên Ngành`: Tên ngành tuyển sinh.

- `	Tổ hợp`: Các tổ hợp môn xét tuyển.

- `	Điểm chuẩn`: Điểm chuẩn tương ứng với tổ hợp.

- `	Ghi chú`: Ghi chú thêm (thường là thông tin về điều kiện bổ sung, hoặc phương thức xét tuyển).

- `	Năm`: Năm tuyển sinh.

#### 3. Mỗi cột hiện có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp cần xử lý tiếp hay không?

In [289]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37399 entries, 0 to 37414
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Tỉnh/ Thành phố      37399 non-null  object 
 1   Khu vực              37399 non-null  object 
 2   Mã trường xét tuyển  37399 non-null  object 
 3   Mã Trường            37399 non-null  object 
 4   Tên Trường           37399 non-null  object 
 5   Mã xét tuyển         37399 non-null  object 
 6   Tên Ngành            37399 non-null  object 
 7   Tổ hợp               37376 non-null  object 
 8   Điểm chuẩn           37399 non-null  float64
 9   Ghi chú              25079 non-null  object 
 10  Năm                  37399 non-null  int64  
 11  Loại đơn vị          37399 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 3.7+ MB


Các cột có kiểu dữ liệu phù hợp.

#### 4. Với mỗi cột, các giá trị (dạng số, dạng phân loại) được phân bố như thế nào?

##### 4.1. Các cột dạng số

In [290]:
numerical_columns = full_df.select_dtypes(include=['int64', 'float64']).columns
print("Numerical columns:", numerical_columns)

def calculate_missing_ratio(df):
    # Tính tỷ lệ dữ liệu thiếu cho từng cột
    missing_ratios = df.isna().mean() * 100

    # Trả về Series chứa tỷ lệ dữ liệu thiếu cho từng cột
    return missing_ratios

# Gọi hàm tính missing ratio và in kết quả
missing_ratios = calculate_missing_ratio(full_df[numerical_columns])
print('Missing ratio:')
print(missing_ratios)
full_df.describe()

Numerical columns: Index(['Điểm chuẩn', 'Năm'], dtype='object')
Missing ratio:
Điểm chuẩn    0.0
Năm           0.0
dtype: float64


,Điểm chuẩn,Năm
count,37399.000000,37399.00000
mean,74.280419,2020.85751
std,183.143994,1.67799
min,0.000000,2018.00000
25%,16.000000,2019.00000
50%,20.000000,2021.00000
75%,25.100000,2022.00000
max,1035.000000,2023.00000


In [291]:
# Tìm và in ra các dòng có dữ liệu thiếu trong cột 'Năm' hoặc cột 'Điểm chuẩn' (nếu có)
missing_rows = full_df[full_df['Năm'].isna() | full_df['Điểm chuẩn'].isna()]
missing_rows

,Tỉnh/ Thành phố,Khu vực,Mã trường xét tuyển,Mã Trường,Tên Trường,Mã xét tuyển,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị


In [292]:
full_df.drop(missing_rows.index, inplace=True)
full_df.reset_index(drop=True, inplace=True)

C:\Users\letra\AppData\Local\Temp\ipykernel_89344\3106419298.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df.drop(missing_rows.index, inplace=True)


In [293]:
# Tìm và in ra các dòng có dữ liệu thiếu trong cột 'Năm' hoặc cột 'Điểm chuẩn' (nếu có)
missing_rows = full_df[full_df['Năm'].isna() | full_df['Điểm chuẩn'].isna()]
missing_rows

,Tỉnh/ Thành phố,Khu vực,Mã trường xét tuyển,Mã Trường,Tên Trường,Mã xét tuyển,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị


Thấy min của điểm chuẩn là 0 (kỳ lạ) nên ta sẽ xử lý các dòng có điểm chuẩn = 0 một chút.


In [294]:
full_df[full_df['Điểm chuẩn']==0]

,Tỉnh/ Thành phố,Khu vực,Mã trường xét tuyển,Mã Trường,Tên Trường,Mã xét tuyển,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị
4104,Hà Nội,Miền Bắc,NHH,BA,Học Viện Ngân Hàng,7340405_J1,Hệ thống thông tin quản lý,"A00, A01, D01, D07",0.0,NaN,2018,Công lập
4105,Hà Nội,Miền Bắc,NHH,BA,Học Viện Ngân Hàng,7340301_J1,Kế toán,"A00, A01, D01, D07",0.0,NaN,2018,Công lập
4106,Hà Nội,Miền Bắc,NHH,BA,Học Viện Ngân Hàng,7380107_A,Luật,"A00, A01, D07, D08",0.0,Luật kinh tế,2018,Công lập
4107,Hà Nội,Miền Bắc,NHH,BA,Học Viện Ngân Hàng,7340201_I,Tài chính - Ngân hàng,"A00, A01, D01, D07",0.0,"Liên kết với Đại học Sunderland, Vương quốc Anh",2018,Công lập
4108,Bắc Ninh,Miền Bắc,DBH,BHIU,Đại Học Quốc Tế Bắc Hà,7480201,Công nghệ thông tin,"A00, A01, A02, A10",0.0,NaN,2018,Dân lập
...,...,...,...,...,...,...,...,...,...,...,...,...
37036,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7520212,Kỹ thuật y sinh,"A00, B00, A01, A02, XDHB",0.0,NaN,2023,Dân lập
37038,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7520320,Kỹ thuật môi trường,"A00, B00, A02, C08, XDHB",0.0,NaN,2023,Dân lập
37040,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7720802,Quản trị bệnh viện,"B00, B03, C01, C02, XDHB",0.0,NaN,2023,Dân lập
37042,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7810202,Quản trị nhà hàng và dịch vụ ăn uống,"A00, A01, D01, C00, XDHB",0.0,NaN,2023,Dân lập


Ta thấy có rất nhiều ngành có điểm chuẩn =0, tuy nhiên, một số sẽ có chú thích riêng. Vậy, đối với các ngành có điểm chuẩn =0 và không có chú thích thì mình xem như năm đó không có tuyển sinh. 
> => Mình loại bỏ các dòng này.

In [295]:
# diemchuan_0 = full_df[(full_df['Điểm chuẩn']==0) & (full_df['Ghi chú'].isna())]
diemchuan_0 = full_df[(full_df['Điểm chuẩn']==0)]
diemchuan_0

,Tỉnh/ Thành phố,Khu vực,Mã trường xét tuyển,Mã Trường,Tên Trường,Mã xét tuyển,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị
4104,Hà Nội,Miền Bắc,NHH,BA,Học Viện Ngân Hàng,7340405_J1,Hệ thống thông tin quản lý,"A00, A01, D01, D07",0.0,NaN,2018,Công lập
4105,Hà Nội,Miền Bắc,NHH,BA,Học Viện Ngân Hàng,7340301_J1,Kế toán,"A00, A01, D01, D07",0.0,NaN,2018,Công lập
4106,Hà Nội,Miền Bắc,NHH,BA,Học Viện Ngân Hàng,7380107_A,Luật,"A00, A01, D07, D08",0.0,Luật kinh tế,2018,Công lập
4107,Hà Nội,Miền Bắc,NHH,BA,Học Viện Ngân Hàng,7340201_I,Tài chính - Ngân hàng,"A00, A01, D01, D07",0.0,"Liên kết với Đại học Sunderland, Vương quốc Anh",2018,Công lập
4108,Bắc Ninh,Miền Bắc,DBH,BHIU,Đại Học Quốc Tế Bắc Hà,7480201,Công nghệ thông tin,"A00, A01, A02, A10",0.0,NaN,2018,Dân lập
...,...,...,...,...,...,...,...,...,...,...,...,...
37036,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7520212,Kỹ thuật y sinh,"A00, B00, A01, A02, XDHB",0.0,NaN,2023,Dân lập
37038,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7520320,Kỹ thuật môi trường,"A00, B00, A02, C08, XDHB",0.0,NaN,2023,Dân lập
37040,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7720802,Quản trị bệnh viện,"B00, B03, C01, C02, XDHB",0.0,NaN,2023,Dân lập
37042,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7810202,Quản trị nhà hàng và dịch vụ ăn uống,"A00, A01, D01, C00, XDHB",0.0,NaN,2023,Dân lập


In [296]:
shape = full_df.shape

# Drop các dòng có trong diemchuan_0
full_df.drop(diemchuan_0.index, inplace=True)

# Reset lại chỉ số sau khi drop
full_df.reset_index(drop=True, inplace=True)

print(f"Dữ liệu trước xử lý có {shape[0]} dòng và {shape[1]} cột.")
print(f"Dữ liệu   sau xử lý có {full_df.shape[0]} dòng và {full_df.shape[1]} cột.")

Dữ liệu trước xử lý có 37399 dòng và 12 cột.
Dữ liệu   sau xử lý có 36583 dòng và 12 cột.


C:\Users\letra\AppData\Local\Temp\ipykernel_89344\1664910539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df.drop(diemchuan_0.index, inplace=True)


##### 4.2. Các cột dạng phân loại

In [297]:
categorical_columns = full_df.select_dtypes(include=['object']).columns
categorical_columns

Index(['Tỉnh/ Thành phố', 'Khu vực', 'Mã trường xét tuyển', 'Mã Trường',
       'Tên Trường', 'Mã xét tuyển', 'Tên Ngành', 'Tổ hợp', 'Ghi chú',
       'Loại đơn vị'],
      dtype='object')

In [298]:
def calculate_column_profile(column):
    missing_ratio = len(column[column.isna()]) * 100 / len(column)
    num_diff_vals = len(column.dropna().unique())
    diff_vals = column.dropna().unique()
    return pd.Series([missing_ratio, num_diff_vals, diff_vals], index=['missing_ratio', 'num_diff_vals', 'diff_vals'])

cate_col_profiles_df = full_df[categorical_columns].agg(calculate_column_profile)
cate_col_profiles_df

,Tỉnh/ Thành phố,Khu vực,Mã trường xét tuyển,Mã Trường,Tên Trường,Mã xét tuyển,Tên Ngành,Tổ hợp,Ghi chú,Loại đơn vị
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.853593,0.0
num_diff_vals,52,3,277,256,289,2567,675,2486,5227,2
diff_vals,"[Hà Nội, Hồ Chí Minh, Bình Định, Vĩnh Phúc, Ng...","[Miền Bắc, Miền Nam, Miền Trung - Tây Nguyên]","[HNM, GNT, MTH, NHF, KHA, SPS, SPK, DNT, BPH, ...","[HMU, NUAE, VUFA, HANU, NEU, HCMUE, HCMUTE, HU...","[Đại học Thủ Đô Hà Nội, Đại Học Sư Phạm Nghệ T...","[7140201, 7210208, 7229042, 7140222, 7210403, ...","[Giáo dục Mầm non, ĐH Piano, Quản lý văn hoá, ...","[M01, M09, N00, R00, H00, Hình hoạ bố cục, D01...","[TTNV ≤ 1Hệ đại học, Năng khiếu nhân 2, Liên t...","[Công lập, Dân lập]"


In [299]:
# Drop rows where 'Ghi chú' contains 'ao đẳng' (case-insensitive)
condition1 = (full_df['Ghi chú'].str.contains('ao đẳng', case=False, na=False))
full_df = full_df.loc[~condition1]

# Drop rows where 'Ghi chú' contains 'CAO ĐẲNG' (case-insensitive)
condition2 = (full_df['Ghi chú'].str.contains('CAO ĐẲNG', case=False, na=False))
full_df = full_df.loc[~condition2]

# Drop rows where 'Mã xét tuyển' contains any of the specified codes
condition3 = (full_df['Mã xét tuyển'].str.contains('51140', na=False) |
              full_df['Mã xét tuyển'].str.contains('52210', na=False) |
              full_df['Mã xét tuyển'].str.contains('51120', na=False) |
              full_df['Mã xét tuyển'].str.contains('52220', na=False) |
              full_df['Mã xét tuyển'].str.contains('52310', na=False) |
              full_df['Mã xét tuyển'].str.contains('52340', na=False) |
              full_df['Mã xét tuyển'].str.contains('52380', na=False) |
              full_df['Mã xét tuyển'].str.contains('52510', na=False) |
              full_df['Mã xét tuyển'].str.contains('51402', na=False) |
              full_df['Mã xét tuyển'].str.contains('521', na=False) |
              full_df['Mã xét tuyển'].str.contains('5620', na=False) |
              full_df['Mã xét tuyển'].str.contains('5340', na=False) |
              full_df['Mã xét tuyển'].str.contains('5480', na=False) |
              full_df['Mã xét tuyển'].str.contains('5510', na=False) |
              full_df['Mã xét tuyển'].str.contains('6210', na=False) |
              full_df['Mã xét tuyển'].str.contains('6220', na=False) |
              full_df['Mã xét tuyển'].str.contains('6510', na=False) |
              full_df['Mã xét tuyển'].str.contains('6340', na=False) |
              full_df['Mã xét tuyển'].str.contains('6480', na=False) |
              full_df['Mã xét tuyển'].str.contains('6540', na=False) |
              full_df['Mã xét tuyển'].str.contains('6620', na=False) |
              full_df['Mã xét tuyển'].str.contains('6640', na=False) |
              full_df['Mã xét tuyển'].str.contains('6760', na=False) |
              full_df['Mã xét tuyển'].str.contains('6810', na=False) |
              full_df['Mã xét tuyển'].str.contains('6320', na=False) |
              full_df['Mã xét tuyển'].str.contains('x', na=False))
full_df = full_df.loc[~condition3]

# Reset lại chỉ số sau khi drop
full_df.reset_index(drop=True, inplace=True)

### **Sửa mã nhóm ngành**

In [300]:
full_df['Mã nhóm ngành'] = full_df['Mã xét tuyển']
change_code_df = pd.read_csv('./data/university/ma_nganh.csv')
# Tạo từ điển tra cứu từ nhom_nganh_df và đảm bảo các khóa đều là chuỗi
ma_nganh_dict = {str(k): v for k, v in zip( change_code_df['Mã ngành'],change_code_df['Mã nhóm ngành'])}

# Hàm để lấy Mã nhóm ngành và Nhóm ngành dựa trên Mã xét tuyển
def processing_manganh(ma_nganh):
    if pd.isna(ma_nganh):
        return None
    if ma_nganh in ma_nganh_dict:
        nhom_nganh = ma_nganh_dict[ma_nganh]
        return nhom_nganh
    else:
        return ma_nganh
    
full_df['Mã nhóm ngành'] = full_df['Mã nhóm ngành'].apply(processing_manganh)
full_df

,Tỉnh/ Thành phố,Khu vực,Mã trường xét tuyển,Mã Trường,Tên Trường,Mã xét tuyển,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị,Mã nhóm ngành
0,Hà Nội,Miền Bắc,HNM,HMU,Đại học Thủ Đô Hà Nội,7140201,Giáo dục Mầm non,"M01, M09",37.58,TTNV ≤ 1Hệ đại học,2018,Công lập,7140201
1,Hà Nội,Miền Bắc,GNT,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7210208,ĐH Piano,N00,32.00,Năng khiếu nhân 2,2018,Công lập,7210208
2,Hà Nội,Miền Bắc,GNT,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7229042,Quản lý văn hoá,R00,32.00,Liên thông (Năng khiếu nhân 2),2018,Công lập,7229042
3,Hà Nội,Miền Bắc,GNT,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7140222,Sư phạm Mỹ thuật,H00,32.00,Liên thông (Năng khiếu nhân 2),2018,Công lập,7140222
4,Hà Nội,Miền Bắc,GNT,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7210403,Thiết kế đồ họa,H00,32.00,Năng khiếu nhân 2,2018,Công lập,7210403
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36128,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,DGNLHCM,761.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,7220209
36129,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,DGNLHCM,648.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,7310601
36130,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,DGNLHCM,643.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,7220203
36131,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,DGNLHCM,613.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,7220202


### **Thêm phân ngành**

In [301]:
phan_nganh_df = pd.read_csv('./data/university/phan_nganh.csv')

# Tạo từ điển tra cứu từ phan_nganh_df và đảm bảo các khóa đều là chuỗi
phan_nganh_dict = {str(k): v for k, v in zip(phan_nganh_df ['Mã phân ngành'], phan_nganh_df['Tên phân ngành'])}

# Hàm để lấy Mã nhóm ngành và Nhóm ngành dựa trên Mã xét tuyển
def get_nhom_nganh(ma_nganh):
    if pd.isna(ma_nganh):
        return pd.Series([None, None])
    ma_phan_nganh = str(ma_nganh)[:5]  # Trích xuất 3 ký tự đầu
    if ma_phan_nganh in phan_nganh_dict:
        phan_nganh = phan_nganh_dict[ma_phan_nganh]
        return pd.Series([ma_phan_nganh, phan_nganh])
    else:
        return pd.Series([None, None])

# Áp dụng hàm để thêm cột Mã nhóm ngành và Nhóm ngành vào full_df
full_df[['Mã phân ngành', 'Phân ngành']] = full_df['Mã nhóm ngành'].apply(get_nhom_nganh)
full_df

,Tỉnh/ Thành phố,Khu vực,Mã trường xét tuyển,Mã Trường,Tên Trường,Mã xét tuyển,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị,Mã nhóm ngành,Mã phân ngành,Phân ngành
0,Hà Nội,Miền Bắc,HNM,HMU,Đại học Thủ Đô Hà Nội,7140201,Giáo dục Mầm non,"M01, M09",37.58,TTNV ≤ 1Hệ đại học,2018,Công lập,7140201,71402,Đào tạo giáo viên
1,Hà Nội,Miền Bắc,GNT,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7210208,ĐH Piano,N00,32.00,Năng khiếu nhân 2,2018,Công lập,7210208,72102,Nghệ thuật trình diễn
2,Hà Nội,Miền Bắc,GNT,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7229042,Quản lý văn hoá,R00,32.00,Liên thông (Năng khiếu nhân 2),2018,Công lập,7229042,72290,Nhân văn - Khác
3,Hà Nội,Miền Bắc,GNT,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7140222,Sư phạm Mỹ thuật,H00,32.00,Liên thông (Năng khiếu nhân 2),2018,Công lập,7140222,71402,Đào tạo giáo viên
4,Hà Nội,Miền Bắc,GNT,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7210403,Thiết kế đồ họa,H00,32.00,Năng khiếu nhân 2,2018,Công lập,7210403,72104,Mỹ thuật ứng dụng
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36128,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,DGNLHCM,761.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,7220209,72202,"Ngôn ngữ, văn học và văn hóa nước ngoài"
36129,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,DGNLHCM,648.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,7310601,73106,Khu vực học
36130,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,DGNLHCM,643.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,7220203,72202,"Ngôn ngữ, văn học và văn hóa nước ngoài"
36131,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,DGNLHCM,613.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,7220202,72202,"Ngôn ngữ, văn học và văn hóa nước ngoài"


### **Thêm nhóm ngành**

In [302]:
nhom_nganh_df = pd.read_csv('./data/university/nhom_nganh.csv')

# Tạo từ điển tra cứu từ nhom_nganh_df và đảm bảo các khóa đều là chuỗi
nhom_nganh_dict = {str(k): v for k, v in zip(nhom_nganh_df['Mã nhóm ngành'], nhom_nganh_df['Nhóm ngành'])}

# Hàm để lấy Mã nhóm ngành và Nhóm ngành dựa trên Mã Ngành
def get_nhom_nganh(ma_nganh):
    if pd.isna(ma_nganh):
        return pd.Series([None, None])
    ma_nhom_nganh = str(ma_nganh)[:3]  # Trích xuất 3 ký tự đầu
    if ma_nhom_nganh in nhom_nganh_dict:
        nhom_nganh = nhom_nganh_dict[ma_nhom_nganh]
        return pd.Series([ma_nhom_nganh, nhom_nganh])
    else:
        return pd.Series([None, None])

# Áp dụng hàm để thêm cột Mã nhóm ngành và Nhóm ngành vào full_df
full_df[['Mã nhóm ngành', 'Nhóm ngành']] = full_df['Mã nhóm ngành'].apply(get_nhom_nganh)
full_df

,Tỉnh/ Thành phố,Khu vực,Mã trường xét tuyển,Mã Trường,Tên Trường,Mã xét tuyển,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị,Mã nhóm ngành,Mã phân ngành,Phân ngành,Nhóm ngành
0,Hà Nội,Miền Bắc,HNM,HMU,Đại học Thủ Đô Hà Nội,7140201,Giáo dục Mầm non,"M01, M09",37.58,TTNV ≤ 1Hệ đại học,2018,Công lập,714,71402,Đào tạo giáo viên,Khoa học giáo dục và đào tạo giáo viên
1,Hà Nội,Miền Bắc,GNT,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7210208,ĐH Piano,N00,32.00,Năng khiếu nhân 2,2018,Công lập,721,72102,Nghệ thuật trình diễn,Nghệ thuật
2,Hà Nội,Miền Bắc,GNT,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7229042,Quản lý văn hoá,R00,32.00,Liên thông (Năng khiếu nhân 2),2018,Công lập,722,72290,Nhân văn - Khác,Nhân văn
3,Hà Nội,Miền Bắc,GNT,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7140222,Sư phạm Mỹ thuật,H00,32.00,Liên thông (Năng khiếu nhân 2),2018,Công lập,714,71402,Đào tạo giáo viên,Khoa học giáo dục và đào tạo giáo viên
4,Hà Nội,Miền Bắc,GNT,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7210403,Thiết kế đồ họa,H00,32.00,Năng khiếu nhân 2,2018,Công lập,721,72104,Mỹ thuật ứng dụng,Nghệ thuật
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36128,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,DGNLHCM,761.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,72202,"Ngôn ngữ, văn học và văn hóa nước ngoài",Nhân văn
36129,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,DGNLHCM,648.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,731,73106,Khu vực học,Khoa học xã hội và hành vi
36130,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,DGNLHCM,643.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,72202,"Ngôn ngữ, văn học và văn hóa nước ngoài",Nhân văn
36131,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,DGNLHCM,613.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,72202,"Ngôn ngữ, văn học và văn hóa nước ngoài",Nhân văn


In [303]:
full_df.to_excel("./data/full/diemchuan_collapsed.xlsx", index=False)

### **Tách cột Tổ hợp**

In [304]:
# Tách cột "Tổ hợp" theo dấu phẩy và tạo các dòng mới
#df_expanded = full_df.assign(subject_group=full_df['Tổ hợp'].str.split(', ')).explode('Tổ hợp')
df_expanded = full_df.assign(subject_group=full_df['Tổ hợp'].str.split(', ')).explode('subject_group')

df_expanded.reset_index(drop=True, inplace=True)
df_expanded

,Tỉnh/ Thành phố,Khu vực,Mã trường xét tuyển,Mã Trường,Tên Trường,Mã xét tuyển,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị,Mã nhóm ngành,Mã phân ngành,Phân ngành,Nhóm ngành,subject_group
0,Hà Nội,Miền Bắc,HNM,HMU,Đại học Thủ Đô Hà Nội,7140201,Giáo dục Mầm non,"M01, M09",37.58,TTNV ≤ 1Hệ đại học,2018,Công lập,714,71402,Đào tạo giáo viên,Khoa học giáo dục và đào tạo giáo viên,M01
1,Hà Nội,Miền Bắc,HNM,HMU,Đại học Thủ Đô Hà Nội,7140201,Giáo dục Mầm non,"M01, M09",37.58,TTNV ≤ 1Hệ đại học,2018,Công lập,714,71402,Đào tạo giáo viên,Khoa học giáo dục và đào tạo giáo viên,M09
2,Hà Nội,Miền Bắc,GNT,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7210208,ĐH Piano,N00,32.00,Năng khiếu nhân 2,2018,Công lập,721,72102,Nghệ thuật trình diễn,Nghệ thuật,N00
3,Hà Nội,Miền Bắc,GNT,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7229042,Quản lý văn hoá,R00,32.00,Liên thông (Năng khiếu nhân 2),2018,Công lập,722,72290,Nhân văn - Khác,Nhân văn,R00
4,Hà Nội,Miền Bắc,GNT,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7140222,Sư phạm Mỹ thuật,H00,32.00,Liên thông (Năng khiếu nhân 2),2018,Công lập,714,71402,Đào tạo giáo viên,Khoa học giáo dục và đào tạo giáo viên,H00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112520,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,DGNLHCM,761.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,72202,"Ngôn ngữ, văn học và văn hóa nước ngoài",Nhân văn,DGNLHCM
112521,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,DGNLHCM,648.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,731,73106,Khu vực học,Khoa học xã hội và hành vi,DGNLHCM
112522,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,DGNLHCM,643.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,72202,"Ngôn ngữ, văn học và văn hóa nước ngoài",Nhân văn,DGNLHCM
112523,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,DGNLHCM,613.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,72202,"Ngôn ngữ, văn học và văn hóa nước ngoài",Nhân văn,DGNLHCM


In [305]:
# # Lọc và xóa các dòng có subject_group là 'XDHB' và 'ọc bạ' tồn tại trong Ghi chú
# condition = (df_expanded['subject_group'] == 'XDHB') & (df_expanded['Ghi chú'].str.contains('Học bạ', na=False))|(df_expanded['subject_group'] == 'XDHB') & (df_expanded['Ghi chú'].str.contains('HỌC BẠ', na=False))
# df_expanded.drop(df_expanded[condition].index, inplace=True)


condition3 = (df_expanded['subject_group'] == 'XDHB') & (df_expanded['Tổ hợp'].str.contains(', XDHB', na=False))
df_expanded.drop(df_expanded[condition3].index, inplace=True)

# Đặt lại chỉ mục của full_df và lưu thay đổi này vào DataFrame gốc
df_expanded.reset_index(drop=True, inplace=True)
df_expanded



,Tỉnh/ Thành phố,Khu vực,Mã trường xét tuyển,Mã Trường,Tên Trường,Mã xét tuyển,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị,Mã nhóm ngành,Mã phân ngành,Phân ngành,Nhóm ngành,subject_group
0,Hà Nội,Miền Bắc,HNM,HMU,Đại học Thủ Đô Hà Nội,7140201,Giáo dục Mầm non,"M01, M09",37.58,TTNV ≤ 1Hệ đại học,2018,Công lập,714,71402,Đào tạo giáo viên,Khoa học giáo dục và đào tạo giáo viên,M01
1,Hà Nội,Miền Bắc,HNM,HMU,Đại học Thủ Đô Hà Nội,7140201,Giáo dục Mầm non,"M01, M09",37.58,TTNV ≤ 1Hệ đại học,2018,Công lập,714,71402,Đào tạo giáo viên,Khoa học giáo dục và đào tạo giáo viên,M09
2,Hà Nội,Miền Bắc,GNT,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7210208,ĐH Piano,N00,32.00,Năng khiếu nhân 2,2018,Công lập,721,72102,Nghệ thuật trình diễn,Nghệ thuật,N00
3,Hà Nội,Miền Bắc,GNT,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7229042,Quản lý văn hoá,R00,32.00,Liên thông (Năng khiếu nhân 2),2018,Công lập,722,72290,Nhân văn - Khác,Nhân văn,R00
4,Hà Nội,Miền Bắc,GNT,NUAE,Đại Học Sư Phạm Nghệ Thuật Trung Ương,7140222,Sư phạm Mỹ thuật,H00,32.00,Liên thông (Năng khiếu nhân 2),2018,Công lập,714,71402,Đào tạo giáo viên,Khoa học giáo dục và đào tạo giáo viên,H00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106278,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,DGNLHCM,761.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,72202,"Ngôn ngữ, văn học và văn hóa nước ngoài",Nhân văn,DGNLHCM
106279,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,DGNLHCM,648.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,731,73106,Khu vực học,Khoa học xã hội và hành vi,DGNLHCM
106280,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,DGNLHCM,643.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,72202,"Ngôn ngữ, văn học và văn hóa nước ngoài",Nhân văn,DGNLHCM
106281,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,DGNLHCM,613.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,72202,"Ngôn ngữ, văn học và văn hóa nước ngoài",Nhân văn,DGNLHCM


In [306]:
categorical_columns = df_expanded.select_dtypes(include=['object']).columns
cate_col_profiles_df = df_expanded[categorical_columns].agg(calculate_column_profile)
cate_col_profiles_df

,Tỉnh/ Thành phố,Khu vực,Mã trường xét tuyển,Mã Trường,Tên Trường,Mã xét tuyển,Tên Ngành,Tổ hợp,Ghi chú,Loại đơn vị,Mã nhóm ngành,Mã phân ngành,Phân ngành,Nhóm ngành,subject_group
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.05953,0.0,0.881609,1.220327,1.220327,0.881609,0.0
num_diff_vals,52,3,276,255,288,2407,661,2451,5135,2,24,83,83,24,234
diff_vals,"[Hà Nội, Hồ Chí Minh, Bình Định, Vĩnh Phúc, Ng...","[Miền Bắc, Miền Nam, Miền Trung - Tây Nguyên]","[HNM, GNT, NHF, KHA, SPS, SPK, DNT, BPH, DQN, ...","[HMU, NUAE, HANU, NEU, HCMUE, HCMUTE, HUFLIT, ...","[Đại học Thủ Đô Hà Nội, Đại Học Sư Phạm Nghệ T...","[7140201, 7210208, 7229042, 7140222, 7210403, ...","[Giáo dục Mầm non, ĐH Piano, Quản lý văn hoá, ...","[M01, M09, N00, R00, H00, D01, C00, D14, D15, ...","[TTNV ≤ 1Hệ đại học, Năng khiếu nhân 2, Liên t...","[Công lập, Dân lập]","[714, 721, 722, 781, 732, 738, 748, 731, 751, ...","[71402, 72102, 72290, 72104, 72202, 78102, 781...","[Đào tạo giáo viên, Nghệ thuật trình diễn, Nhâ...","[Khoa học giáo dục và đào tạo giáo viên, Nghệ ...","[M01, M09, N00, R00, H00, D01, C00, D14, D15, ..."


In [307]:
# df_expanded.to_csv("./data/preprocessing/full.csv",  encoding='utf-8-sig', index=False)

In [308]:
# Tạo cột 'Loại điểm'
df_expanded['Loại điểm'] = ''
# Loại học bạ
df_expanded.loc[(df_expanded['Ghi chú']=='Xét học bạ')|(df_expanded['Ghi chú'].str.contains('Học bạ', na=False))|(df_expanded['Tổ hợp'].str.contains('XDHB', na=False)), 'Loại điểm'] = 'Học bạ'

# Cập nhật 'Loại điểm' DGNL
df_expanded.loc[(df_expanded['Tổ hợp'].str.contains('DGNLQGHN', na=False)),  'Loại điểm'] = 'DGNLQGHN'
df_expanded.loc[(df_expanded['Tổ hợp'].str.contains('DGTD', na=False))| df_expanded['Ghi chú'].str.contains('DGTD', na=False), 'Loại điểm'] = 'DGTD'
df_expanded.loc[(df_expanded['Tổ hợp'] == 'DGNLHCM') | ((df_expanded['Tổ hợp'] == 'DGNL') & (df_expanded['Điểm chuẩn'] >= 400)), 'Loại điểm'] = 'DGNLHCM'
df_expanded.loc[(df_expanded['Ghi chú'].str.contains('Xét kết hợp điểm hợp ĐGNL, TN THPT, và học bạ', na=False))|
                (df_expanded['Ghi chú'].str.contains('Tốt nghiệp THPT; Điểm thi TN THPT theo tổ hợp xét tuyển x 3', na=False)),  'Loại điểm'] = 'BKHCM'

df_expanded.loc[df_expanded['Loại điểm'] == '', 'Loại điểm'] = 'THPTQG'
# df_expanded

In [309]:
df_expanded.drop(columns = ['Tổ hợp'], inplace=True)
df_expanded.rename(columns={'subject_group': 'Tổ hợp'}, inplace=True)
df_expanded = df_expanded[['Tỉnh/ Thành phố', 'Khu vực', 
                           'Mã trường xét tuyển', 'Mã Trường', 'Tên Trường', 'Loại đơn vị', 
                           'Mã nhóm ngành', 'Nhóm ngành', 'Mã phân ngành', 'Phân ngành', 
                           'Mã xét tuyển', 'Tên Ngành', 'Loại điểm', 'Tổ hợp', 'Điểm chuẩn', 'Ghi chú', 'Năm']]
# df_expanded 

In [310]:
df_expanded.to_excel("./data/diemchuan_full.xlsx", index=False)